In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.nonparametric import smoothers_lowess
from pandas import Series, DataFrame
from patsy import dmatrices
from sklearn import datasets, svm
# import xgboost as xgb


In [4]:
inputfile = "rawData/train.csv"
traindata = pd.read_csv(inputfile)


In [12]:
inputfile2 = "rawData/test.csv"
testdata = pd.read_csv(inputfile2)


In [13]:
testdata.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
df_col = traindata.columns

In [6]:
df_col

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [10]:
train_x = traindata.loc[:,['Fare']]

In [8]:
train_y = traindata.loc[:,['Survived']]

In [9]:
train_x.head(5)

,Pclass
0,3
1,1
2,3
3,1
4,3


In [11]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_x, train_y)

In [14]:
test_x = testdata.loc[:,['Fare']]

In [16]:
test_x = test_x.replace(np.NAN,0)

In [17]:
test_y = clf.predict(test_x)

In [20]:
test_y[:5]

array([0, 0, 0, 0, 1])

In [22]:
# pd.DataFrame(test_y).to_csv("rawData/submission.csv",index=False)

In [26]:
outputdf = pd.DataFrame(test_y,columns=['Survived'])

In [27]:
outputdf.head(5)

,Survived
0,0
1,0
2,0
3,0
4,1


In [30]:
outputdf2 = testdata.loc[:,['PassengerId']].merge(outputdf, left_index=True, right_index=True)

In [31]:
outputdf2.head(5)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [33]:
outputdf2.to_csv("submission.csv",index=False)

In [34]:
tt = outputdf2.merge(outputdf2, left_index=True, right_index=True)

In [35]:
tt.head()

,PassengerId_x,Survived_x,PassengerId_y,Survived_y
0,892,0,892,0
1,893,0,893,0
2,894,0,894,0
3,895,0,895,0
4,896,1,896,1
